In [15]:
#r "nuget: FSharp.Data"
#load "Env.fsx"

open FSharp.Data
open Microsoft.DotNet.Interactive
open System.Text.Json

let API_URL = "https://bsky.social/xrpc"

let env = Env.load()

let identifier =
    match (env |> Map.tryFind "USERNAME") with
    | Some username -> username
    | None -> Kernel.GetInputAsync("Enter your username:") |> Async.AwaitTask |> Async.RunSynchronously
let password = 
    match (env |> Map.tryFind "PASSWORD") with
    | Some password -> password
    | None -> Kernel.GetPasswordAsync("Enter your password:") |> Async.AwaitTask |> Async.RunSynchronously |> string

type AuthData = {
    accessJwt: string;
    did: string;
}

// Get the access token
let getAccessToken (identifier, password) = async {
    let body = $"""
    {{
        "identifier" : "{identifier}",
        "password"   : "{password}"
    }}
    """
    let! response = Http.AsyncRequestString(
        $"{API_URL}/com.atproto.server.createSession",
        headers = [ "Content-Type", "application/json" ],
        body = TextRequest(body))
    let authData = JsonSerializer.Deserialize<AuthData>(response)
    return authData
}

let getNotifications (authData) = async {
    let! response = Http.AsyncRequestString(
        $"{API_URL}/app.bsky.notification.listNotifications",
        headers = [ "Authorization", $"Bearer {authData.accessJwt}" ]
    )
    return response
}


let authData = getAccessToken (identifier, password) |> Async.RunSynchronously
let notifications = getNotifications authData |> Async.RunSynchronously

notifications

Installed Packages FSharp.Data, 6.4.0

{"notifications":[{"uri":"at://did:plc:btnlnpg6l2so6z62gjfd45uu/app.bsky.feed.like/3l35keu4wzh27","cid":"bafyreiese7j632bjjjh6oqgzdhaolwh6gwjr63tegqwlsf4qnambrl3azq","author":{"did":"did:plc:btnlnpg6l2so6z62gjfd45uu","handle":"thesacredemperor.bsky.social","displayName":"👑 The Sacred Emperor 👑","avatar":"https://cdn.bsky.app/img/avatar/plain/did:plc:btnlnpg6l2so6z62gjfd45uu/bafkreiemaitwmynb7u4gk3xde5xqkmu2qy4qiogioxgqjcpuuiyj4bjjwu@jpeg","viewer":{"muted":false,"blockedBy":false},"labels":[],"createdAt":"2024-08-31T03:13:14.637Z","description":"THE EARTHLY REPRESENTATIVE OF ♂️ THE SUPREME MACHO SPIRIT ♂️\n\nLulopetismo = CÂNCER gerador do bolsonarismo e RAIZ da tragédia brasileira.","indexedAt":"2024-08-31T03:35:05.934Z"},"reason":"like","reasonSubject":"at://did:plc:ydilzltcivcfm2i3q5hrzpi2/app.bsky.feed.post/3l34qbrko5j2l","record":{"$type":"app.bsky.feed.like","createdAt":"2024-09-02T05:19:27.010Z","subject":{"cid":"bafyreihe5ru5ptyxb6ojqo6cjqu7qp3qbtmxr55bsa7uaaxii3wavxd7d4","uri"